## Import Libraries

This code imports libraries to process data with pandas, convert text into TF-IDF vectors, and measure text similarity using cosine similarity.



In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

## Define Data Path

This code defines the variable DATA_PATH to store the file path of the dataset (Dataset.csv).


In [2]:
DATA_PATH = "/content/Dataset .csv"

## Load and Preprocess Data

This code loads a CSV into a DataFrame, cleans column names, and creates a new profile column by combining cuisines, city, locality, and price range into a single text field.

In [3]:
df = pd.read_csv(r"E:\Backup\Mahima\ML Internship 2025\Dataset.csv")
df.columns = [c.strip().lower().replace(" ", "_") for c in df.columns]
df["profile"] = (df["cuisines"].fillna('') + " | " +
                 df["city"].fillna('') + " | " +
                 df["locality"].fillna('') + " | " +
                 df["price_range"].astype(str))

## Display Head of DataFrame

This code displays the first five rows of the DataFrame df.


In [4]:
df.head()

,restaurant_id,restaurant_name,country_code,city,address,locality,locality_verbose,longitude,latitude,cuisines,...,has_table_booking,has_online_delivery,is_delivering_now,switch_to_order_menu,price_range,aggregate_rating,rating_color,rating_text,votes,profile
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Yes,No,No,No,3,4.8,Dark Green,Excellent,314,"French, Japanese, Desserts | Makati City | Cen..."
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Yes,No,No,No,3,4.5,Dark Green,Excellent,591,"Japanese | Makati City | Little Tokyo, Legaspi..."
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Yes,No,No,No,4,4.4,Green,Very Good,270,"Seafood, Asian, Filipino, Indian | Mandaluyong..."
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,No,No,No,No,4,4.9,Dark Green,Excellent,365,"Japanese, Sushi | Mandaluyong City | SM Megama..."
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Yes,No,No,No,4,4.8,Dark Green,Excellent,229,"Japanese, Korean | Mandaluyong City | SM Megam..."


## Create TF-IDF Vectorizer and Matrix

This code converts the profile column into a TF-IDF feature matrix X using TfidfVectorizer.

In [5]:
tfidf = TfidfVectorizer(token_pattern=r"[A-Za-z0-9_]+")
X = tfidf.fit_transform(df["profile"].astype(str))

## Define Recommendation Function

This function generates restaurant recommendations by matching user preferences to the dataset using TF-IDF vectors and cosine similarity, returning the top k similar results.

In [6]:
def recommend(user_prefs, top_k=5):
    query = " | ".join([str(v) for v in user_prefs.values() if v])
    q_vec = tfidf.transform([query])
    sims = cosine_similarity(q_vec, X).ravel()
    top_idx = sims.argsort()[::-1][:top_k]
    return df.iloc[top_idx][["restaurant_name","city","locality","cuisines","aggregate_rating"]]

## Test Recommendation Function

This code prints the top 5 restaurant recommendations in Makati City serving French cuisine based on similarity scores.

In [7]:
print(recommend({"cuisines":"French","city":"Makati City"}, top_k=5))

                  restaurant_name             city  \
0                Le Petit Souffle      Makati City   
1                Izakaya Kikufuji      Makati City   
3257                 Bonne Bouche        New Delhi   
9329  Hippopotamus - Museum Hotel  Wellington City   
460                          Jaan        Singapore   

                                        locality                    cuisines  \
0      Century City Mall, Poblacion, Makati City  French, Japanese, Desserts   
1     Little Tokyo, Legaspi Village, Makati City                    Japanese   
3257                              Defence Colony             Italian, French   
9329                                      Te Aro                French, Kiwi   
460                     City Hall, Downtown Core                      French   

      aggregate_rating  
0                  4.8  
1                  4.5  
3257               4.1  
9329               4.4  
460                3.8  
